In [ ]:
!pip install matplotlib seaborn 
# pandas numpy scipy

Import Librerie

In [1]:
import pandas as pd
import numpy as np
from functools import reduce

Caricamento Dataset con pandas

In [2]:
data = pd.read_csv("./data_csv/atp_matches_qual_chall_2022.csv", sep=",")
data_2 = pd.read_csv("./data_csv/atp_matches_qual_chall_2023.csv", sep=",")
data_3 = pd.read_csv("./data_csv/atp_matches_qual_chall_2024.csv", sep=",")

In [3]:
# Concatenate DataFrames vertically
df = pd.concat([data, data_2, data_3], axis=0, ignore_index=True)

In [ ]:
df.head()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31869 entries, 0 to 31868
Data columns (total 49 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tourney_id          31869 non-null  object 
 1   tourney_name        31869 non-null  object 
 2   surface             31869 non-null  object 
 3   draw_size           31869 non-null  int64  
 4   tourney_level       31869 non-null  object 
 5   tourney_date        31869 non-null  int64  
 6   match_num           31869 non-null  int64  
 7   winner_id           31869 non-null  int64  
 8   winner_seed         15895 non-null  float64
 9   winner_entry        6186 non-null   object 
 10  winner_name         31869 non-null  object 
 11  winner_hand         31869 non-null  object 
 12  winner_ht           31298 non-null  float64
 13  winner_ioc          31869 non-null  object 
 14  winner_age          31862 non-null  float64
 15  loser_id            31869 non-null  int64  
 16  lose

1.data → Il DataFrame di partenza.

2.['winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'] → Elenco delle colonne da rimuovere.

3.axis=1 → Specifica che si vogliono rimuovere le colonne (non righe).

4.inplace=True → Apporta le modifiche direttamente al DataFrame senza dover riassegnarlo.

In [ ]:
#df.drop(['match_num','winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'], axis=1, inplace=True) # da inserire in una tabella a parte insieme a winner_id e loser_id

#ranking_player = df[['winner_id','loser_id','winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points']].copy()

# con ranking possiamo capire se il giocatore dato per favorito perde (con ranking loser > ranking winner), 
# frequenza, in quale stagione ecc..


Considerazione: score è una colonna che, se il mio obiettivo è trovare il giocatore che ha maggior probabilità di vincere è fondamentale perchè posso capire diversi fattori e trasformali in medie:
1. Calcolare con tutto il data, quante volte si è ritirato considerando il numero di partite totali e le partite W/O diamo un peso, mentre RET un'altro media ponderata.
    - in che stagione si è ritirato di più
    - su che tipo di campo
    - frequenza di ritiro 1 volta ogni 7 partite, ecc..
    - quando si è ritirato di più best of di 3 o 5
    - a che round 
    - dopo quanti minuti
    - Dopo però nelle successive analisi posso eliminarla W/O e RET
2. 7-6(5) 7-6(1) quante volte il giocatore va al tiè break e quante volte vince/perde
    - devo separare ogni valore presente nelle righe e separarlo in colonne 'score_1set_w', 'score_2set_l'
    - quindi nel primo set tende a superare 5 se vince, viceversa se perde non  fa <=3
    - se torneo meglio 3 o meglio 5 quanti punti fa, fa spesso 2 a 0
3. Formattare data
    - Trovare stagioni
4. Prendere 2 giocatori che hanno giocato più partite e fare breve analisi sulle prestazioni (magari anche testa a testa)
    - Prendere ogni torneo e fare analisi sulle prestazioni generali

L'obiettivo è creare delle classi o funzioni che fanno questi punti. Di modo che alla fine si fa un main.py e faremo girare l'app demo in streamlit, dove possiamo inserire un id o nome (giocatore o torneo) e lui ci farà tutte le analisi del caso. Difficile ma carino.

Vedere se i tornei sono tutti a eliminazione diretta!


1. Cancellare RET e W/O
2. dividere score
3. formattare data, stagione 





In [ ]:
#for score in data['score'].unique():
#    print(score)

#num_wo = np.sum(data['score'].values == 'W/O')
#num_wo

#num_ret = np.sum(np.char.find(data['score'].values.astype(str), 'RET') >= 0)
#num_ret

#num_true = np.sum(data['ritiro'].values)
#num_true

In [ ]:
df['ritiro'] = np.logical_or(
    np.char.find(df['score'].values.astype(str), 'RET') >= 0,
    np.char.find(df['score'].values.astype(str), 'W/O') >= 0
)

In [ ]:
#calcolo_ritiro = df[['winner_id','winner_name','loser_id', 'loser_name', 'score', 'ritiro', 'best_of', 'round', 'minutes']].copy()

In [ ]:
#calcolo_ritiro.head()

In [ ]:
#calcolo_ritiro.info()

In [24]:
#ritiro_rows = df[np.char.find(df['score'].values.astype(str), 'RET') >= 0]
#ritiro_rows

#ritiro_rows = df[np.char.find(df['score'].values.astype(str), 'W/O') >= 0]
#ritiro_rows

#ritiro_rows = df[np.char.find(df['score'].values.astype(str), 'DEF') >= 0]
#ritiro_rows['score']

ritiro_rows = df[np.char.find(df['score'].values.astype(str), 'DEF') >= 0]
ritiro_rows


#num_wo = np.sum(calcolo_ritiro['score'].values == 'W/O')
#num_wo

for score in df['score'].unique():
    print(score)

6-2 6-1
5-7 7-5 6-2
6-4 6-3
6-1 6-1
6-0 6-1
6-1 3-6 6-4
6-4 1-6 6-1
6-4 6-4
4-6 6-3 6-2
7-6(5) 7-6(1)
6-4 6-2
6-7(4) 6-2 6-3
6-7(4) 6-4 6-4
6-4 3-6 6-4
7-5 6-3
4-6 6-1 7-6(5)
6-2 6-3
7-6(7) 6-2
6-1 2-6 6-4
6-2 6-2
6-7(2) 7-6(3) 6-1
6-2 5-7 6-3
6-3 6-1
7-6(6) 6-3
6-3 6-2
6-3 6-3
3-6 6-3 6-4
7-5 6-2
7-5 6-4
6-3 3-6 6-1
6-4 1-6 6-3
4-6 6-4 6-3
6-3 7-6(4)
2-6 7-6(6) 6-4
6-1 7-5
6-7(3) 6-4 6-2
6-7(2) 7-6(4) 7-5
7-5 6-1
6-3 7-6(5)
6-2 6-4
6-2 7-6(2)
7-6(5) 5-7 7-6(5)
6-3 4-6 6-3
7-6(7) 6-3
2-6 6-4 7-6(5)
7-5 4-6 6-1
6-3 3-6 6-0
7-6(3) 6-7(4) 7-5
2-6 6-3 7-6(4)
7-6(7) 6-4
7-6(2) 6-3
6-3 6-4
6-7(3) 6-1 6-3
6-4 7-5
7-6(1) 6-3
4-6 7-6(1) 6-2
7-5 6-7(3) 7-6(3)
7-6(3) 6-1
6-2 2-6 7-5
6-3 3-6 7-5
7-6(2) 5-7 6-3
6-7(6) 6-4 6-4
6-4 6-0
6-4 4-6 6-3
5-7 6-3 6-2
6-4 7-6(6)
7-5 6-7(4) 6-3
6-3 1-6 6-3
2-6 6-4 6-4
4-6 6-4 7-5
6-4 2-6 6-4
6-4 3-6 6-3
2-6 6-3 6-3
6-7(2) 6-4 7-5
6-0 6-3
6-4 6-1
6-1 6-2
6-4 7-6(3)
1-6 6-3 6-3
6-2 7-6(4)
7-6(0) 3-6 6-1
6-1 7-6(4)
2-6 6-4 6-3
6-7(5) 6-4 6-1
7-6(3) 6-3
6-7(4) 6-4

### Alla fine salvare in una tabella calcolo_ritiro

In [ ]:
eliminare = reduce(np.logical_or, [
    np.char.find(df['score'].values.astype(str), 'RET') >= 0,
    np.char.find(df['score'].values.astype(str), 'W/O') >= 0,
    np.char.find(df['score'].values.astype(str), 'DEF') >= 0,
    np.char.find(df['score'].values.astype(str), 'Ret') >= 0,
    np.char.find(df['score'].values.astype(str), 'Def.') >= 0
])

df = df[~eliminare]

In [ ]:
# Cancellare False da calcolo ritiro
#calcolo_ritiro = calcolo_ritiro[calcolo_ritiro['ritiro'].values]

In [28]:
# Funzione per separare i set e gestire i tie-break
def extract_scores(score):
    try:
        sets = score.split()
        scores = []
        
        for s in sets:
            if '(' in s:  # Caso Tie-Break
                try:
                    w, tiebreak = s.split('(')
                    w, l = w.split('-')
                    scores.extend([int(w), int(l), int(tiebreak.strip(')'))])
                except ValueError:
                    scores.extend([np.nan, np.nan, np.nan])
            elif '-' in s:
                try:
                    w, l = s.split('-')
                    scores.extend([int(w), int(l), np.nan])  # No tie-break
                except ValueError:
                    scores.extend([np.nan, np.nan, np.nan])    
            else:
                scores.extend([np.nan, np.nan, np.nan])
            
        # Completamento con NaN se mancano dati
        while len(scores) < 15:
            scores.append(np.nan)
        
        return scores[:15]

    except Exception as e:
        print(f"Errore con il valore '{score}': {e}")
        return [np.nan] * 15  # Riempie i valori con NaN in caso di errore



In [ ]:
# Applicazione della funzione e creazione delle colonne
score_cols = ['score_1set_w', 'score_1set_l', 'score_tiebreak_1set_l', 
              'score_2set_w', 'score_2set_l', 'score_tiebreak_2set_l',
              'score_3set_w', 'score_3set_l', 'score_tiebreak_3set_l',
              'score_4set_w', 'score_4set_l', 'score_tiebreak_4set_l',
              'score_5set_w', 'score_5set_l', 'score_tiebreak_5set_l']

df[score_cols] = np.array([extract_scores(s) for s in df['score']])



      tourney_id  tourney_name surface  draw_size tourney_level  tourney_date  \
0      2022-0059    Salinas CH    Hard         32             C      20220404   
1      2022-0059    Salinas CH    Hard         32             C      20220404   
2      2022-0059    Salinas CH    Hard         32             C      20220404   
3      2022-0059    Salinas CH    Hard         32             C      20220404   
4      2022-0059    Salinas CH    Hard         32             C      20220404   
...          ...           ...     ...        ...           ...           ...   
31864  2024-8268  Yokkaichi CH    Hard         32             C      20241125   
31865  2024-8268  Yokkaichi CH    Hard         32             C      20241125   
31866  2024-8268  Yokkaichi CH    Hard         32             C      20241125   
31867  2024-8268  Yokkaichi CH    Hard         32             C      20241125   
31868  2024-8268  Yokkaichi CH    Hard         32             C      20241125   

       match_num  winner_id